# 06wk-1: (신경망) – 다중클래스분류

최규빈  
2025-04-02

<a href="https://colab.research.google.com/github/guebin/DL2025/blob/main/posts/05wk-1
    .ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# 1. 강의영상

<https://youtu.be/playlist?list=PLQqh36zP38-yx6DQsLACqw8pWm0udv8Jm&si=in1eMD0-wU49y7mS>

# 2. Imports

In [5]:
import torch
import matplotlib.pyplot as plt

In [6]:
plt.rcParams['figure.figsize'] = (4.5, 3.0)

# 5. 이진분류 코딩패턴

In [199]:
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True)
to_tensor = torchvision.transforms.ToTensor()
X0_train = torch.stack([to_tensor(Xi) for Xi, yi in train_dataset if yi==0])
X1_train = torch.stack([to_tensor(Xi) for Xi, yi in train_dataset if yi==1])
X0_test = torch.stack([to_tensor(Xi) for Xi, yi in test_dataset if yi==0])
X1_test = torch.stack([to_tensor(Xi) for Xi, yi in test_dataset if yi==1])
X = torch.concat([X0_train,X1_train],axis=0).reshape(-1,784)
y = torch.tensor([0.0]*len(X0_train) + [1.0]*len(X1_train)).reshape(-1,1)
XX = torch.concat([X0_test,X1_test],axis=0).reshape(-1,784)
yy = torch.tensor([0.0]*len(X0_test) + [1.0]*len(X1_test)).reshape(-1,1)

## A. 시그모이드 수동구현

In [271]:
torch.manual_seed(0)
net = torch.nn.Sequential(
    torch.nn.Linear(784,32),
    torch.nn.ReLU(),
    torch.nn.Linear(32,1)
)
optimizr = torch.optim.Adam(net.parameters())

In [272]:
for epoc in range(100):
    # step1 
    netout = net(X) 
    yhat = torch.exp(netout) / (1 + torch.exp(netout))
    # step2
    loss = loss_fn(yhat,y) # loss_fn = torch.nn.BCELoss()
    # step3     
    loss.backward()
    # step4 
    optimizr.step()
    optimizr.zero_grad()

In [266]:
(((net(XX) > 0) == yy)*1.0).mean()

## B. `BCEWithLogitsLoss`

In [375]:
torch.manual_seed(0)
net = torch.nn.Sequential(
    torch.nn.Linear(784,32),
    torch.nn.ReLU(),
    torch.nn.Linear(32,1),
)
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizr = torch.optim.Adam(net.parameters())

In [376]:
for epoc in range(100):
    # step1 
    netout = net(X)
    # step2
    loss = loss_fn(netout,y)
    # step3     
    loss.backward()
    # step4 
    optimizr.step()
    optimizr.zero_grad()

In [377]:
(((net(XX) > 0) == yy)*1.0).mean()

## C. `log_sigmoid`

In [378]:
torch.manual_seed(0)
net = torch.nn.Sequential(
    torch.nn.Linear(784,32),
    torch.nn.ReLU(),
    torch.nn.Linear(32,1),
)
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizr = torch.optim.Adam(net.parameters())

In [379]:
for epoc in range(100):
    # step1 
    netout = net(X)
    # step2
    loss = loss_fn(netout,y)
    # step3     
    loss.backward()
    # step4 
    optimizr.step()
    optimizr.zero_grad()

In [377]:
(((net(XX) > 0) == yy)*1.0).mean()

In [381]:
torch.exp(torch.nn.functional.logsigmoid(netout))

In [386]:
torch.nn.functional.nll_loss(torch.exp(torch.nn.functional.logsigmoid(netout)),y.flatten().long())

## C. 과잉모수

In [302]:
torch.manual_seed(0)
net = torch.nn.Sequential(
    torch.nn.Linear(784,32),
    torch.nn.ReLU(),
    torch.nn.Linear(32,2),
)
bce = torch.nn.BCEWithLogitsLoss()
ce = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(net.parameters())

In [303]:
netout = net(X)

In [304]:
1 / (1+torch.exp(-netout)) # 시도1

In [345]:
yhat = torch.exp(netout)/ (torch.exp(netout).sum(axis=1)).reshape(-1,1)
yhat

In [309]:
yhat

In [374]:
net = torch.nn.Sequential(
    torch.nn.Linear(784,32),
    torch.nn.ReLU(),
    torch.nn.Linear(32,2),
)
bce = torch.nn.BCELoss()
ce = torch.nn.CrossEntropyLoss()
netout = net(X)
yhat = torch.exp(netout)/ (torch.exp(netout).sum(axis=1)).reshape(-1,1)
print((bce(yhat[:,[0]],y) + bce(yhat[:,[1]],(y == 0)*1.0))/2)
#print(ce(netout,torch.nn.functional.one_hot(y.flatten().long()).float()))
print(ce(netout,y.flatten().long()))

tensor(0.6868, grad_fn=<DivBackward0>)
tensor(0.7154, grad_fn=<NllLossBackward0>)

In [364]:
torch.nn.functional.one_hot(y.flatten().long()).float()

In [363]:
netout

In [176]:
for epoc in range(100):
    # step1 
    netout = net(X)
    # step2
    loss = loss_fn(yhat,y)
    # step3     
    loss.backward()
    # step4 
    optimizr.step()
    optimizr.zero_grad()

In [182]:
((net(XX).argmax(axis=1) == yy)*1.0).mean()

# 5. 다중클래스 분류

## A. 결론 (그냥 외우세요)

`-` 2개의 class를 구분하는 문제가 아니라 $k$개의 class를 구분해야
한다면?

***일반적인 개념***

-   손실함수: BCE loss $\to$ Cross Entropy loss
-   마지막층의 선형변환: torch.nn.Linear(?,1) $\to$ torch.nn.Linear(?,k)
-   **마지막층의 활성화: sig $\to$ softmax**

***파이토치 한정***

-   **y의형태: (n,) vector + int형 // (n,k) one-hot encoded matrix +
    float형**
-   손실함수: torch.nn.BCEWithLogitsLoss, $\to$
    torch.nn.CrossEntropyLoss
-   마지막층의 선형변환: torch.nn.Linear(?,1) $\to$ torch.nn.Linear(?,k)
-   **마지막층의 활성화: None $\to$ None (손실함수에 이미 마지막층의
    활성화가 포함)**

## B. 실습: 3개의 클래스를 구분

`-` 정리된 코드1: 통계잘하는데 파이토치 못쓰는 사람의 코드

In [ ]:
## Step1: 데이터준비 
path = fastai.data.external.untar_data('https://s3.amazonaws.com/fast-ai-imageclas/mnist_png.tgz')
X0 = torch.stack([torchvision.io.read_image(str(fname)) for fname in (path/'training/0').ls()])
X1 = torch.stack([torchvision.io.read_image(str(fname)) for fname in (path/'training/1').ls()])
X2 = torch.stack([torchvision.io.read_image(str(fname)) for fname in (path/'training/2').ls()])
X = torch.concat([X0,X1,X2]).reshape(-1,1*28*28)/255
y = torch.nn.functional.one_hot(torch.tensor([0]*len(X0) + [1]*len(X1)+ [2]*len(X2))).float()
## Step2: 학습가능한 오브젝트 생성
torch.manual_seed(43052)
net = torch.nn.Sequential(
    torch.nn.Linear(784,32),
    torch.nn.ReLU(),
    torch.nn.Linear(32,3),
#    torch.nn.Softmax()
)
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(net.parameters())
## Step3: 적합 
for epoc in range(100):
    ## step1 
    netout = net(X)
    ## step2 
    loss = loss_fn(netout,y)
    ## step3 
    loss.backward()
    ## step4 
    optimizr.step()
    optimizr.zero_grad()
    
## Step4: 적합 (혹은 적합결과확인)
(netout.argmax(axis=1) == y.argmax(axis=1)).float().mean()

`-` 정리된 코드2: 파이토치를 잘하는 사람의 코드

In [ ]:
## Step1: 데이터준비 
path = fastai.data.external.untar_data('https://s3.amazonaws.com/fast-ai-imageclas/mnist_png.tgz')
X0 = torch.stack([torchvision.io.read_image(str(fname)) for fname in (path/'training/0').ls()])
X1 = torch.stack([torchvision.io.read_image(str(fname)) for fname in (path/'training/1').ls()])
X2 = torch.stack([torchvision.io.read_image(str(fname)) for fname in (path/'training/2').ls()])
X = torch.concat([X0,X1,X2]).reshape(-1,1*28*28)/255
#y = torch.nn.functional.one_hot(torch.tensor([0]*len(X0) + [1]*len(X1)+ [2]*len(X2))).float()
y = torch.tensor([0]*len(X0) + [1]*len(X1)+ [2]*len(X2))
## Step2: 학습가능한 오브젝트 생성
torch.manual_seed(43052)
net = torch.nn.Sequential(
    torch.nn.Linear(784,32),
    torch.nn.ReLU(),
    torch.nn.Linear(32,3),
#    torch.nn.Softmax()
)
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(net.parameters())
## Step3: 적합 
for epoc in range(100):
    ## step1 
    netout = net(X)
    ## step2 
    loss = loss_fn(netout,y)
    ## step3 
    loss.backward()
    ## step4 
    optimizr.step()
    optimizr.zero_grad()
## Step4: 적합 (혹은 적합결과확인)    
(netout.argmax(axis=1) == y).float().mean()

-   완전같은코드임

## C. Softmax

`-` 눈치: softmax를 쓰기 직전의 숫자들은 (n,k)꼴로 되어있음. 각
observation 마다 k개의 숫자가 있는데, 그중에서 유난히 큰 하나의 숫자가
있음.

In [ ]:
net(X)

In [ ]:
y

`-` 수식

-   $\text{sig}(u)=\frac{e^u}{1+e^u}$
-   $\text{softmax}({\boldsymbol u})=\text{softmax}([u_1,u_2,\dots,u_k])=\big[ \frac{e^{u_1}}{e^{u_1}+\dots e^{u_k}},\dots,\frac{e^{u_k}}{e^{u_1}+\dots e^{u_k}}\big]$

`-` torch.nn.Softmax() 손계산

(예시1) – 잘못계산

In [64]:
softmax = torch.nn.Softmax(dim=0)

In [65]:
netout = torch.tensor([[-2.0,-2.0,0.0],
                        [3.14,3.14,3.14],
                        [0.0,0.0,2.0],
                        [2.0,2.0,4.0],
                        [0.0,0.0,0.0]])
netout

In [66]:
softmax(netout) 

(예시2) – 이게 맞게 계산되는 것임

In [67]:
softmax = torch.nn.Softmax(dim=1)

In [68]:
netout

In [69]:
softmax(netout)

(예시3) – 차원을 명시안하면 맞게 계산해주고 경고 줌

In [70]:
softmax = torch.nn.Softmax()

In [71]:
netout

In [72]:
softmax(netout)

/home/cgb3/anaconda3/envs/dl2025/lib/python3.9/site-packages/torch/nn/modules/module.py:1739: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)

(예시4) – 진짜 손계산

In [73]:
netout 

In [74]:
torch.exp(netout)

In [75]:
0.1353/(0.1353 + 0.1353 + 1.0000), 0.1353/(0.1353 + 0.1353 + 1.0000), 1.0000/(0.1353 + 0.1353 + 1.0000) # 첫 obs

In [76]:
torch.exp(netout[1])/torch.exp(netout[1]).sum() # 두번째 obs 

## D. CrossEntropyLoss

`-` 수식

***`# 2개의 카테고리`***

`-` 예제1: BCELoss vs BCEWithLogisticLoss

In [77]:
y = torch.tensor([0,0,1]).reshape(-1,1).float()
netout = torch.tensor([-1, 0, 1]).reshape(-1,1).float()
y,netout

In [78]:
# 계산방법1: 공식암기
sig = torch.nn.Sigmoid()
yhat = sig(netout)
- torch.sum(torch.log(yhat)*y + torch.log(1-yhat)*(1-y))/3

In [79]:
# 계산방법2: torch.nn.BCELoss() 이용
sig = torch.nn.Sigmoid()
yhat = sig(netout)
loss_fn = torch.nn.BCELoss()
loss_fn(yhat,y)

In [80]:
# 계산방법3: torch.nn.BCEWithLogitsLoss() 이용
loss_fn = torch.nn.BCEWithLogitsLoss()
loss_fn(netout,y)

`-` 예제2: BCEWithLogisticLoss vs CrossEntropyLoss

In [81]:
torch.concat([sig(netout),1-sig(netout)],axis=1)

In [82]:
netout = torch.tensor([[3,2],[2,2],[5,6]]).float()
y = torch.tensor([[1,0],[1,0],[0,1]]).float()
y,netout #,netout[:,[1]]-netout[:,[0]]

In [83]:
softmax(netout)

In [84]:
# 계산방법1: 공식암기
-torch.sum(torch.log(softmax(netout))*y)/3

In [85]:
# 계산방법2: torch.nn.CrossEntropyLoss() 이용 + y는 one-hot으로 정리
loss_fn = torch.nn.CrossEntropyLoss()
loss_fn(netout,y)

In [86]:
# 계산방법3: torch.nn.CrossEntropyLoss() 이용 + y는 0,1 로 정리
loss_fn = torch.nn.CrossEntropyLoss()
loss_fn(netout,y)

`#`

***`# 3개의 카테고리`***

In [87]:
y = torch.tensor([2,1,2,2,0])
y_onehot = torch.nn.functional.one_hot(y)
netout = torch.tensor(
    [[-2.0000, -2.0000,  0.0000],
     [ 3.1400,  3.1400,  3.1400],
     [ 0.0000,  0.0000,  2.0000],
     [ 2.0000,  2.0000,  4.0000],
     [ 0.0000,  0.0000,  0.0000]]
)
y,y_onehot

In [88]:
## 방법1 -- 추천X
loss_fn = torch.nn.CrossEntropyLoss()
loss_fn(netout,y_onehot.float())

In [89]:
## 방법2 -- 추천O
loss_fn = torch.nn.CrossEntropyLoss()
loss_fn(netout,y)

In [90]:
## 방법3 -- 공식.. (이걸 쓰는사람은 없겠지?)
softmax = torch.nn.Softmax() 
loss_fn = torch.nn.CrossEntropyLoss()
- torch.sum(torch.log(softmax(netout))*y_onehot)/5

`#`

`-` 계산하는 공식을 아는것도 중요한데 torch.nn.CrossEntropyLoss() 에는
softmax 활성화함수가 이미 포함되어 있다는 것을 확인하는 것이 더 중요함.

`-` torch.nn.CrossEntropyLoss() 는 사실 torch.nn.CEWithSoftmaxLoss()
정도로 바꾸는 것이 더 말이 되는 것 같다.

## E. Minor Topic: 이진분류와 CrossEntropy

`-` 2개의 클래스일경우에도 CrossEntropy를 쓸 수 있지 않을까?

In [91]:
## Step1: 데이터준비 
path = fastai.data.external.untar_data(fastai.data.external.URLs.MNIST)
X0 = torch.stack([torchvision.io.read_image(str(fname)) for fname in (path/'training/0').ls()])
X1 = torch.stack([torchvision.io.read_image(str(fname)) for fname in (path/'training/1').ls()])
X = torch.concat([X0,X1]).reshape(-1,1*28*28)/255
y = torch.tensor([0]*len(X0) + [1]*len(X1))
## Step2: 학습가능한 오브젝트 생성
torch.manual_seed(43052)
net = torch.nn.Sequential(
    torch.nn.Linear(784,32),
    torch.nn.ReLU(),
    torch.nn.Linear(32,2),
    #torch.nn.Softmax()
)
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(net.parameters())
## Step3: fit  
for epoc in range(70): 
    ## 1 
    ## 2 
    loss= loss_fn(net(X),y) 
    ## 3 
    loss.backward()
    ## 4 
    optimizr.step()
    optimizr.zero_grad() 
## Step4: Predict 
softmax = torch.nn.Softmax()
(net(X).argmax(axis=1) == y).float().mean()

`-` 이진분류문제 = “y=0 or y=1” 을 맞추는 문제 = 성공과 실패를 맞추는
문제 = 성공확률과 실패확률을 추정하는 문제

`-` softmax, sigmoid

-   softmax: (실패확률, 성공확률) 꼴로 결과가 나옴 // softmax는
    실패확률과 성공확률을 둘다 추정한다.
-   sigmoid: (성공확률) 꼴로 결과가 나옴 // sigmoid는 성공확률만
    추정한다.

`-` 그런데 “실패확률=1-성공확률” 이므로 사실상 둘은 같은걸 추정하는
셈이다. (성공확률만 추정하면 실패확률은 저절로 추정되니까)

`-` 즉 아래는 같은 표현력을 가진 모형이다.

![](https://raw.githubusercontent.com/guebin/DL2024/cdbdf23589efc2198260ab9c749f1757f67a128d/posts/05wk-1-fig2.svg)

![](https://raw.githubusercontent.com/guebin/DL2024/cdbdf23589efc2198260ab9c749f1757f67a128d/posts/05wk-1-fig1.svg)

`-` 둘은 같은 표현력을 가진 모형인데 학습할 파라메터는 sigmoid의 경우가
더 적다. $\to$ sigmoid를 사용하는 모형이 비용은 싸고 효과는 동일하다는
말 $\to$ 이진분류 한정해서는 softmax를 쓰지말고 sigmoid를 써야함.

-   softmax가 갑자기 너무 안좋아보이는데 sigmoid는 k개의 클래스로 확장이
    불가능한 반면 softmax는 확장이 용이하다는 장점이 있음.

## F. 정리

`-` 결론

1.  소프트맥스는 시그모이드의 확장이다.
2.  클래스의 수가 2개일 경우에는 (Sigmoid, BCEloss) 조합을 사용해야 하고
    클래스의 수가 2개보다 클 경우에는 (Softmax, CrossEntropyLoss) 를
    사용해야 한다.

`-` 그런데 사실.. 클래스의 수가 2개일 경우일때 (Softmax,
CrossEntropyLoss)를 사용해도 그렇게 큰일나는것은 아니다. (그냥 좀
비효율적인 느낌이 드는 것 뿐임. 흑백이미지를 칼라잉크로 출력하는 느낌)

***참고***

|               $y$               |            분포가정             | 마지막층의 활성화함수 |   손실함수    |
|:-------------------------------:|:-------------------------------:|:---------------------:|:-------------:|
|     3.45, 4.43, … (연속형)      |            정규분포             |  None (or Identity)   |      MSE      |
|             0 or 1              | 이항분포 with $n=1$ (=베르누이) |        Sigmoid        |      BCE      |
| \[0,0,1\], \[0,1,0\], \[1,0,0\] |       다항분포 with $n=1$       |        Softmax        | Cross Entropy |